In [72]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime
# !pip freeze > requirements.txt

In [ ]:
def fetch_sp500_data(start_date, end_date):
    sp500 = yf.download('^GSPC', start=start_date, end=end_date, auto_adjust=True)
    sp500['Log Returns'] = np.log(sp500['Close'] / sp500['Close'].shift(1))
    sp500['Realized Volatility'] = sp500['Log Returns'].rolling(window=21).std() * np.sqrt(252)
    sp500 = sp500[['Close', 'Log Returns', 'Realized Volatility']]
    sp500.index = pd.to_datetime(sp500.index)  
    # print(sp500.head())  
    return sp500

def fetch_vix_iv(start_date, end_date):
    vix = yf.download('^VIX', start=start_date, end=end_date, auto_adjust=True)
    vix['Implied Volatility'] = vix['Close'] / 100  # VIX index to IV approximation
    vix = vix[['Close', 'Implied Volatility']]
    vix.index = pd.to_datetime(vix.index)  
    # print(vix.head())  
    return vix

def fetch_risk_free_rate(start_date, end_date):
    rf = pdr.get_data_fred('DGS3MO', start_date, end_date)
    rf = rf / 100  
    rf = rf.rename(columns={'DGS3MO': 'Risk-Free Rate'})
    rf.index = pd.to_datetime(rf.index) 
    rf = rf.asfreq('B').fillna(method='ffill')  # Aligneed with business days
    # print(rf.head())  
    return rf

def preprocess_and_merge(sp500, vix, rf):
    # Merge datasets ensuring same index format

    # sp500 = sp500.droplevel(1) if isinstance(sp500.index, pd.MultiIndex) else sp500
    # vix = vix.droplevel(1) if isinstance(vix.index, pd.MultiIndex) else vix
    # rf = rf.droplevel(1) if isinstance(rf.index, pd.MultiIndex) else rf

    # data = sp500.merge(vix, on = "Date", how='inner')
    data = pd.concat([sp500, vix], axis=1, join='inner')
    # print(data.head())  
    
    data.rename(columns={'index': 'Date'}, inplace=True)
    # missing_dates = rf.index.difference(sp500.index)
    # print("Missing dates in risk-free rate:", missing_dates)
    
    rf1 = rf.reindex(sp500.index, method='ffill')

    # data = data.merge(rf1, on = "Date", how='inner')
    data = pd.concat([data, rf1], axis=1, join='inner')
    data.dropna(inplace=True)
    return data

In [ ]:
# date range
start_date = '2015-01-01'
end_date = '2025-01-01'

sp500_data = fetch_sp500_data(start_date, end_date)
vix_data = fetch_vix_iv(start_date, end_date)
risk_free_rate = fetch_risk_free_rate(start_date, end_date)

final_data = preprocess_and_merge(sp500_data, vix_data, risk_free_rate)

final_data.to_csv("sp500_vix_data.csv")

print("Dataset saved as sp500_vix_data.csv")
print(final_data.head())


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Dataset saved as sp500_vix_data.csv
            (Close, ^GSPC)  (Log Returns, )  (Realized Volatility, )  \
Date                                                                   
2015-02-03     2050.030029         0.014336                 0.180978   
2015-02-04     2041.510010        -0.004165                 0.169203   
2015-02-05     2062.520020         0.010239                 0.168782   
2015-02-06     2055.469971        -0.003424                 0.165366   
2015-02-09     2046.739990        -0.004256                 0.153966   

            (Close, ^VIX)  (Implied Volatility, )  Risk-Free Rate  
Date                                                               
2015-02-03      17.330000                  0.1733          0.0002  
2015-02-04      18.330000                  0.1833          0.0001  
2015-02-05      16.850000                  0.1685          0.0002  
2015-02-06      17.290001                  0.1729          0.0002  
2015-02-09      18.549999                  0.1855  


C:\Users\ARSH\AppData\Local\Temp\ipykernel_62348\3064411312.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rf = rf.asfreq('B').fillna(method='ffill')  # Align with business days


In [62]:
df1_columns = pd.MultiIndex.from_tuples([("A0", "B0", "C0"), ("A1", "B1", "C1")])
df1 = pd.DataFrame([[1, 2], [10, 20]], columns=df1_columns)

df2_columns = pd.MultiIndex.from_tuples([("X0", "Y0"), ("X1", "Y1")])
df2 = pd.DataFrame([[1, 200], [10, 200]], columns=df2_columns)

print("DataFrame 1:")
print(df1)
print("DataFrame 2:")
print(df2)

# Concatenate DataFrames with MultiIndex columns
df_combined = pd.concat([df1, df2], axis=1)
print("Combined DataFrame:")
print(df_combined)

print(df1.index)
print(df2.index)
print(df_combined.index)

DataFrame 1:
   A0  A1
   B0  B1
   C0  C1
0   1   2
1  10  20
DataFrame 2:
   X0   X1
   Y0   Y1
0   1  200
1  10  200
Combined DataFrame:
   (A0, B0, C0)  (A1, B1, C1)  (X0, Y0)  (X1, Y1)
0             1             2         1       200
1            10            20        10       200
RangeIndex(start=0, stop=2, step=1)
RangeIndex(start=0, stop=2, step=1)
RangeIndex(start=0, stop=2, step=1)
